In [ ]:
import requests
import openai
import time
from typing import List, Dict

In [ ]:
# Set your OpenAI API key here
openai.api_key = 'your-openai-api-key'

In [ ]:
# 1. Query Semantic Scholar
# -----------------------
def search_papers(query: str, limit: int = 5) -> List[Dict]:
    url = f"https://api.semanticscholar.org/graph/v1/paper/search?query={query}&limit={limit}&fields=title,abstract,year,authors,url"

    # Implement a retry mechanism with exponential backoff
    for i in range(3):  # Retry up to 3 times
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
            papers = response.json().get('data', [])
            return papers
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:  # If rate limited
                wait_time = 2 ** i  # Exponential backoff: 1, 2, 4 seconds
                print(f"Rate limited, retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise  # Re-raise other HTTP errors

    # If all retries fail, raise an exception
    raise Exception("Failed to retrieve papers after multiple retries.")

In [ ]:
!pip install --upgrade openai

In [ ]:
# 2. Summarize Abstracts
# -----------------------
def summarize_abstracts(papers: List[Dict]) -> str:
    abstracts = "\n\n".join(
        [f"Title: {p['title']}\nAbstract: {p['abstract']}" for p in papers if p.get('abstract')]
    )

    if not abstracts:
        return "No abstracts found to summarize."

    prompt = f"""
You are a research assistant. Summarize the key trends, findings, and challenges from the following academic paper abstracts:

{abstracts}

Format the response as:
- Key Trends:
- Major Findings:
- Open Challenges:
"""

    # Updated to use openai.Completion.create with the chat format
    response = openai.Completion.create(
        model="gpt-3.5-turbo",  # or any other suitable chat model
        messages=[
            {"role": "system", "content": "You are a helpful academic assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5
    )
    return response['choices'][0]['text'].strip() # Accessing 'text' instead of 'message'

In [ ]:
# 3. Main Pipeline
# -----------------------
def run_agent(query: str):
    print(f"Searching for papers on: {query}\n")
    papers = search_papers(query)

    for i, paper in enumerate(papers, 1):
        print(f"[{i}] {paper['title']} ({paper['year']})")
        print(f"    Authors: {', '.join([a['name'] for a in paper['authors']])}")
        print(f"    URL: {paper['url']}\n")

    summary = summarize_abstracts(papers)
    print("\n===== Coherent Summary =====\n")
    print(summary)

In [ ]:
# Run Example
# -----------------------
if __name__ == '__main__':
    research_query = input("Enter your research query: ")
    run_agent(research_query)

Enter your research query: research_query
Searching for papers on: research_query


===== Coherent Summary =====

No abstracts found to summarize.
